In [ ]:
#-*- coding: utf-8 -*-
%matplotlib inline
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
import csv
import os
import glob
import math
import time as tm

In [ ]:
# フォントインポート
from matplotlib.font_manager import FontProperties
fp = FontProperties(fname=font_path_you_want, size=text_size)
import matplotlib as mpl
font_paths = mpl.font_manager.findSystemFonts()
font_objects = mpl.font_manager.createFontList(font_paths)
font_names = [f.name for f in font_objects]
'IPAexGothic' in font_names
sns.set('talk', 'darkgrid', 'husl',font='IPAexGothic', font_scale=font_scale)

In [ ]:
class ShowGraphs:
    def plot_distance_graph_from_distance_csv(data_path, xname, xs, yname, ys, figsize, save_path=None):
        files = []
        files = sorted(glob.glob(data_path))
        exclude_actions = ['fall', 'back','sit', 'standfall', 'other', 'stand', 'side', 'bodychange',  'no']
        files_excluded = FileOperation.set_file_excluded(files, exclude_actions)
        stds = np.linspace(0, 0.1, 11)

        fig = plt.figure(figsize = (15,30)) #横サイズ， 縦サイズ
        plt.subplots_adjust(wspace=0.3, hspace=1.0)
        fig.text(0.5, 0.05, 'Time [s]', ha='center', va='center', size=30)
        fig.text(0.02, 0.5, 'Estimated Distance [m]', ha='center', va='center', rotation='vertical', size=30)
        
        for i, file in enumerate(files_excluded):
            title = FileOperation.get_action(file)
            distance = pd.read_csv(str(file), skiprows= 0, header=None, names=['distance'], engine='python')['distance']
            time = np.linspace(start = 0, stop = len(distance)/27.0, num = len(distance) )
                #グラフ1個についての設定
            ax1 = fig.add_subplot(13, 3 , i+1)
    #         ax1 = fig.add_subplot(1, 1, 1)
            plt.plot(time, distance)
            plt.title(title)
            ax1.set_ylim(-0.2 ,3.0)
            plt.tick_params(labelsize=18)
    
    def plot_distance_from_IQ(IQ_path, color, xname, yname, xlim, ylim, figsize, text_size, font_scale, save_path=None):
        #一個の生データから距離グラフを生成
        ##シリアルポート通信用
        IQ_file = IQ_path
        makeFeaturesFile =  MakeFeaturesFile(0, 0, time_diff=0.3, t_observe=2.4, t_division=0.6)
        doppler = makeFeaturesFile._doppler_from_IQdata(IQ_path)
        Spec_raw = makeFeaturesFile._spec_tdp_from_doppler(doppler)[0:, dmin_index[0]:dmax_index[3]]
        Spec = Spec_raw
        
        distance = makeFeaturesFile._max_distance_from_spec(Spec)
        time = np.arange(len(distance))/27.0
        
        plt.rcParams['font.family'] = 'Times New Roman' #全体のフォントを設定
        fig = plt.figure(figsize = figsize)
        plt.plot(time, distance+0.1, color=color)
        plt.xlim(xlim)
        plt.ylim(ylim)
        fig.text(0.5, 0.02, xname, ha='center', va='center', size=text_size)
        fig.text(0.02, 0.55, yname, ha='center', va='center', rotation='vertical', size=text_size)
        if save_path:
            plt.tight_layout()
            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
        
    def show_distance_statistics(IQ_path):
        IQ_file = IQ_path
        makeFeaturesFile =  MakeFeaturesFile(0, 0, time_diff=0.3, t_observe=2.4, t_division=0.6)
        doppler = makeFeaturesFile._doppler_from_IQdata(IQ_path)
        Spec_raw = makeFeaturesFile._spec_tdp_from_doppler(doppler)[0:, dmin_index[0]:dmax_index[3]]
        Spec = Spec_raw
        distance = makeFeaturesFile._max_distance_from_spec(Spec)
        display(pd.DataFrame(pd.Series(distance.ravel()).describe()).transpose())
            
    def show_fft(data_path, color, xname, yname, ax_size, start, end, figsize, save_path=None):
        data_path = data_path
        title = data_path.rsplit('/', 1)[1].split('_', 1)[1].split('.', 1)[0]
        IQdata = pd.read_csv(data_path, delimiter="   ", skiprows= 7, skipfooter=1, header=None, names=['No.', 'I','Q'], engine='python')
        wave = (IQdata.I+1j* IQdata.Q)#doppler信号
        wave_extract = wave[start*5120:end*5120]
        fft_result = np.fft.fft(wave_extract, NFFT) #fft
        
        plt.rcParams['font.family'] = 'Times New Roman' #全体のフォントを設定
        fig = plt.figure(figsize = figsize)
        plt.plot(d, fft_result, color=color)
        plt.xlim(0, 2.4)
        plt.ylim(0, max(fft_result[d<=2.5])+100)
        plt.rcParams['font.family'] = 'Times New Roman' #全体のフォントを設定
        plt.rcParams['axes.xmargin'] = 0.01
        plt.rcParams['axes.ymargin'] = 0.01
        plt.xlabel(xname, fontsize = ax_size)
        plt.ylabel(yname, fontsize = ax_size)
        plt.tick_params(labelsize=18)
        if save_path:
            plt.tight_layout()
            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
        
    def show_spec(Spec, Spec_max, xname, yname, figsize, vmin=None, vmax=None, save_path=None, title=None):
        fig = plt.figure(figsize = figsize)
        Sp_max = Spec_max
        spec_len = np.shape(Spec)[0]
        time = np.linspace(start = 0, stop = spec_len/27.0, num = spec_len )
    
        Y = np.log10(Spec.real.T / Sp_max.real)
        
        df = pd.DataFrame(Y, index=d[dmin_index[0]:dmax_index[3]], columns= time)
        ax1 = fig.add_subplot(1, 1, 1)
        im = ax1.pcolor(time, d[dmin_index[0]:dmax_index[3]], df, cmap="jet", vmin=vmin, vmax=vmax)
        ax1.set_xlim(0, max(time))
        ax1.set_ylim(0 ,2.6)
        ax1.set_xlabel(xname)
        ax1.set_ylabel(yname)
        ax1.set_title(title)
        fig.colorbar(im, ax=ax1)
        if save_path:
            plt.tight_layout()
            plt.savefig(save_path)

    def plot_something_by_something(grahp_type, bar_width, xname, xs, xlim, yname, ys, ylim, figsize, text_size, font_scale, save_path):
        xs = xs
        ys = ys
        fig = plt.figure(figsize = figsize)
        if graph_type == "bar":
            for x, y in zip(xs, ys):
                plt.text(x, y, y, ha='center', va='bottom', size=text_size)
                plt.bar(x, y, color="r", width=bar_width)
            plt.xlim(xlim)
            plt.ylim(ylim)
        else:
            plt.plot(xs, ys)
        plt.tight_layout()
        plt.rcParams['font.family'] = 'Times New Roman' #全体のフォントを設定
        plt.rcParams['axes.xmargin'] = 0.2
        plt.rcParams['axes.ymargin'] = 0.2
        fig.text(0.5, 0.03, xname, ha='center', va='center')
        fig.text(0.03, 0.55, yname, ha='center', va='center', rotation='vertical')
        plt.savefig(save_path)